In [2]:
import pandas as pd
import numpy as np
from data.get_data import FetchPlansPricings
from data.clean_data import HMOEPOCleaner, PPOPOSCleaner
from features.feature_building import FeatureEngineer
from models.v1_model import MedicalPlanSimilarityModel
from sklearn.neighbors import NearestNeighbors
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows', None)

### Fetch Data

In [2]:
data_fetcher = FetchPlansPricings(state="CA", zip_code=95030, year=2023, quarter="Q1")
data_fetcher.fetch_and_save_data()
raw_plans_df = pd.read_csv("~/like-plans/data/raw/raw_plans.csv")

/Users/kieranshaw/like-plans/.conda/lib/python3.10/site-packages/google/cloud/bigquery/_pandas_helpers.py:189: UserWarning: Unable to determine type for field '_airbyte_meta'.
  warnings.warn("Unable to determine type for field '{}'.".format(bq_field.name))


Saved dataframe from CA, 95030, 2023, Q1 to /Users/kieranshaw/like-plans/data/raw/raw_plans.csv
Saved dataframe from CA, 95030, 2023, Q1 to /Users/kieranshaw/like-plans/data/raw/raw_pricings.csv


### Clean Data

In [3]:
# Create an instance of HMOEPOCleaner
hmo_epo_cleaner = HMOEPOCleaner()
ppo_pos_cleaner = PPOPOSCleaner()

# Read the raw data
hmo_epo_raw_data = hmo_epo_cleaner.read_data()
ppo_pos_raw_data = ppo_pos_cleaner.read_data()

# Clean the data
hmo_epo_cleaned_data = hmo_epo_cleaner.clean(df=hmo_epo_raw_data)
ppo_pos_cleaned_data = ppo_pos_cleaner.clean(df=ppo_pos_raw_data)

# Save the cleaned data for inspection
hmo_epo_cleaner.save_data(df=hmo_epo_cleaned_data)
ppo_pos_cleaner.save_data(df=ppo_pos_cleaned_data)

### Feature Building

In [4]:
fe_hmo = FeatureEngineer(exclude_cols=["carrier_name","name","id"])
hmo_epo_cleaned_data["primary_care_physician_in_network"] = hmo_epo_cleaned_data.apply(
    lambda row: fe_hmo.calculate_primary_care_physician(
        row['pcp_cleaned_dollar_values_in_network'], 
        row['pcp_cleaned_percentages_in_network'], 
        row['pcp_initial_visits_in_network']
    ), axis=1
)
hmo_epo_df = fe_hmo.drop_hmo_columns(df=hmo_epo_cleaned_data)
hmo_epo_df.reset_index(drop=True,inplace=True)
hmo_epo_transformed_array = fe_hmo.fit_transform(df=hmo_epo_df)

fe_ppo = FeatureEngineer(exclude_cols=["carrier_name","name","id"])
ppo_pos_cleaned_data["primary_care_physician_in_network"] = ppo_pos_cleaned_data.apply(
    lambda row: fe_hmo.calculate_primary_care_physician(
        row['pcp_cleaned_dollar_values_in_network'], 
        row['pcp_cleaned_percentages_in_network'], 
        row['pcp_initial_visits_in_network']
    ), axis=1
)
ppo_pos_cleaned_data["primary_care_physician_out_of_network"] = ppo_pos_cleaned_data.apply(
    lambda row: fe_hmo.calculate_primary_care_physician(
        row['pcp_cleaned_dollar_values_out_of_network'], 
        row['pcp_cleaned_percentages_out_of_network'], 
        row['pcp_initial_visits_out_of_network']
    ), axis=1
)
ppo_pos_df = fe_hmo.drop_ppo_columns(df=ppo_pos_cleaned_data)
ppo_pos_df.reset_index(drop=True,inplace=True)

ppo_pos_transformed_array = fe_ppo.fit_transform(df=ppo_pos_df)

### Run Model

In [8]:
model = MedicalPlanSimilarityModel(ppo_pos_df, ppo_pos_transformed_array, metric='manhattan', n_neighbors=20)
random_plan, similar_plans = model.fit()

In [9]:
random_plan

,id,carrier_name,name,level,plan_type,hsa_eligible,infertility_treatment_rider,network_size,individual_medical_deductible_in_network,individual_medical_deductible_out_of_network,family_medical_deductible_in_network,family_medical_deductible_out_of_network,individual_medical_moop_in_network,individual_medical_moop_out_of_network,family_medical_moop_in_network,family_medical_moop_out_of_network,coinsurance_in_network,coinsurance_out_of_network,individual_drug_deductible_in_network,family_drug_deductible_in_network,pcp_after_deductible_in_network,pcp_after_deductible_out_of_network,primary_care_physician_in_network,primary_care_physician_out_of_network
327,37873CA0090001-13,UnitedHealthcare,Select Plus PPO HDHP Silver CVQQ /L46S,silver,PPO,1,0,89334,2800,5600,3000,6000,7500,14700,15000,29400,40,50,0,0,1,1,68.4,85.5


In [10]:
similar_plans

,id,carrier_name,name,level,plan_type,hsa_eligible,infertility_treatment_rider,network_size,individual_medical_deductible_in_network,individual_medical_deductible_out_of_network,family_medical_deductible_in_network,family_medical_deductible_out_of_network,individual_medical_moop_in_network,individual_medical_moop_out_of_network,family_medical_moop_in_network,family_medical_moop_out_of_network,coinsurance_in_network,coinsurance_out_of_network,individual_drug_deductible_in_network,family_drug_deductible_in_network,pcp_after_deductible_in_network,pcp_after_deductible_out_of_network,primary_care_physician_in_network,primary_care_physician_out_of_network,similarity_score
235,37873CA0090006-13,UnitedHealthcare,Core PPO HDHP Silver CVQD /L46S,silver,PPO,1,0,56190,2800,5600,3000,6000,7500,14700,15000,29400,40,50,0,0,1,1,68.40,85.5,1.359143
345,70285CA8160433,BlueShield of California,Silver Full PPO Savings 2600/35% HDHP PrevRx O...,silver,PPO,1,0,63389,2600,5200,3000,6000,7500,15000,15000,30000,35,50,0,0,1,1,59.85,85.5,7.851901
371,70285CA8160517,BlueShield of California,Silver Tandem PPO Savings 2600/35% HDHP PrevRx...,silver,PPO,1,0,52961,2600,5200,3000,6000,7500,15000,15000,30000,35,50,0,0,1,1,59.85,85.5,8.279524
326,37873CA0090055,UnitedHealthcare,Select Plus PPO HDHP Bronze CVQY /L45Y,expanded_bronze,PPO,1,0,89334,6000,12000,12000,24000,7500,14700,15000,29400,40,50,0,0,1,1,68.40,85.5,9.061767
347,67138CA0730159,Health Net,Silver HDHP PPO 1500/50%,silver,PPO,1,0,67835,1500,3000,3000,6000,7000,14000,14000,28000,50,50,0,0,1,1,85.50,85.5,9.377996
348,67138CA0730159-33,Health Net,Silver HDHP PPO 1500/50% + Chiropractic,silver,PPO,1,0,67835,1500,3000,3000,6000,7000,14000,14000,28000,50,50,0,0,1,1,85.50,85.5,9.377996
343,70285CA8160011,BlueShield of California,Silver Full PPO Savings 2300/25% OffEx,silver,PPO,1,0,63389,2300,4600,3000,6000,7500,15000,15000,30000,25,50,0,0,1,1,42.75,85.5,9.388099
46,27603CA0000002-77,CalChoice,Anthem Prudent Buyer Silver PPO D (OOA),silver,PPO,1,0,90435,2000,4000,4000,8000,7050,14100,14100,28200,35,50,0,0,1,1,59.85,85.5,9.776963
55,27603CA1221547,Anthem,Anthem Silver PPO 2600/35% w/HSA PrevRx,silver,PPO,1,0,90435,2600,5200,5200,10400,7050,14100,14100,28200,35,50,0,0,1,1,59.85,85.5,9.777204
369,70285CA8160382,BlueShield of California,Silver Tandem PPO Savings 2300/25% OffEx,silver,PPO,1,0,52961,2300,4600,3000,6000,7500,15000,15000,30000,25,50,0,0,1,1,42.75,85.5,9.815722


### Random

In [16]:
raw_plans_df = pd.read_csv("~/like-plans/data/raw/raw_plans.csv")
raw_plans_df[raw_plans_df["network_size"] == 0]["name"]

220         CA Bronze HMO $75/125 7900
221    CA Bronze HMO $75/125 7900 wINF
242             CA Gold HMO $25/50 500
243        CA Gold HMO $25/50 500 wINF
244               CA Gold HMO $30/60 0
245          CA Gold HMO $30/60 0 wINF
246            CA Gold HMO $30/70 1250
247       CA Gold HMO $30/70 1250 wINF
248             CA Gold HMO $35/65 250
249        CA Gold HMO $35/65 250 wINF
290           CA Platinum HMO $20/40 0
291      CA Platinum HMO $20/40 0 wINF
306             CA Silver HMO $50/70 0
307        CA Silver HMO $50/70 0 wINF
308         CA Silver HMO $60/100 2500
309    CA Silver HMO $60/100 2500 wINF
Name: name, dtype: object

In [22]:
# raw_plans_df[raw_plans_df["chiropractic_services"].isna()]['name']
raw_plans_df[(raw_plans_df["id"] == "20523CA0031474") | (raw_plans_df["id"] == "20523CA0031072")]["id"]

5     20523CA0031072
19    20523CA0031474
Name: id, dtype: object

In [5]:
raw_plans_df[raw_plans_df["adult_dental"].isna()]["name"][802]

'Silver Tandem PPO Savings 2600/35% HDHP PrevRx OffEx'

In [59]:
raw_plans_df[raw_plans_df["name"] == "Anthem Bronze PPO 75/7300/40%"]["id"]

232    27603CA1221529
Name: id, dtype: object

In [25]:
raw_plans_df[raw_plans_df["chiropractic_services"].isna()][["id","name","carrier_name","benefits_summary_url"]]

,id,name,carrier_name,benefits_summary_url
100,27603CA1210266-13,Anthem Gold Select HMO 35 w/ T&L,Anthem,NaN
102,27603CA1210290-13,Anthem Gold HMO 35 w/ T&L,Anthem,NaN
104,27603CA1210475-13,Anthem Platinum Select HMO 0/25 w/ T&L,Anthem,NaN
106,27603CA1210510-13,Anthem Platinum HMO 0/25 w/ T&L,Anthem,NaN
108,27603CA1210534-13,Anthem Gold Select HMO 30 w/ T&L,Anthem,NaN
110,27603CA1210551-13,Anthem Gold HMO 30 w/ T&L,Anthem,NaN
112,27603CA1210773-13,Anthem Silver HMO 55 w/ T&L,Anthem,NaN
114,27603CA1210792-13,Anthem Silver Select HMO 55 w/ T&L,Anthem,NaN
116,27603CA1210865-13,Anthem Platinum HMO 0/30 w/ T&L,Anthem,NaN
118,27603CA1210884-13,Anthem Platinum Select HMO 0/30 w/ T&L,Anthem,NaN


In [30]:
len(raw_plans_df)

902